In [10]:
import ipywidgets.widgets as widgets
from IPython.display import display
image_display = widgets.Image(format="jpeg", width="45%")
full_display = widgets.Image(format="jpeg", width="45%")
display(widgets.HBox([image_display, full_display]))

location_coords_display = widgets.Label()
display(location_coords_display)

import cv2
import pyzed.sl as sl
camera = sl.Camera()
camera_params = sl.InitParameters()
camera_params.camera_resolution = sl.RESOLUTION.VGA
camera_params.depth_mode = sl.DEPTH_MODE.ULTRA
camera_params.coordinate_units = sl.UNIT.MILLIMETER

camera_status = camera.open(camera_params)
if camera_status != sl.ERROR_CODE.SUCCESS:
    print("camera error")
    print(camera_status)
    camera.close()
    exit()

# initialize robot
import motors
robot = motors.MotorsYukon()
robot.stop()

# initialize model
import tracking
m = tracking.Model()

# get initial image and choose object to track
image_mat = sl.Mat()
started_tracking = False
while not started_tracking:
    err = camera.grab()
    if err == sl.ERROR_CODE.SUCCESS:
        camera.retrieve_image(image_mat)
        image = image_mat.get_data()
        
        image_display.value = m.show_all_boxes(image)
    
        user_input = input("enter id to track (or leave blank to skip):")
        if user_input == "":
            continue
        else:
            m.tracked_id = int(user_input)
            print("tracking object id " + str(user_input))
            started_tracking = True

# turning parameters
KEEP_X_BETWEEN = (250, 432)
TURN_SPEED = 0.3

# start tracking
running = True
while running:
    err = camera.grab()
    if err == sl.ERROR_CODE.SUCCESS:
        camera.retrieve_image(image_mat)
        image = image_mat.get_data()

        tracked_box = m.track(image, return_type="corners")
        x_centre = int((tracked_box[0] + tracked_box[2]) / 2)
        print("horizontal centre: " + str(x_centre))
        
        if tracked_box is False:
            image_display.value = bytes(cv2.imencode('.jpg', image)[1])
            robot.stop()
        else:
            image_rect = cv2.rectangle(
                image,
                (int(tracked_box[0]), int(tracked_box[1])),
                (int(tracked_box[2]), int(tracked_box[3])),
                (255, 0, 0),
                4
            )
            image_display.value = bytes(cv2.imencode('.jpg', image_rect)[1])

            # turn if tracked person is outside KEEP_X_BETWEEN
            if x_centre < KEEP_X_BETWEEN[0]:
                robot.left(speed=TURN_SPEED)
            elif x_centre > KEEP_X_BETWEEN[1]:
                robot.right(speed=TURN_SPEED)
            else:
                robot.stop()

        full_display.value = m.show_all_boxes(image)
    

camera.close()

Label(value='')

[2025-04-30 11:15:10 UTC][ZED][INFO] Logging level INFO
[2025-04-30 11:15:10 UTC][ZED][INFO] Logging level INFO
[2025-04-30 11:15:10 UTC][ZED][INFO] Logging level INFO
[2025-04-30 11:15:11 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-04-30 11:15:11 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-04-30 11:15:11 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-04-30 11:15:11 UTC][ZED][INFO] [Init]  Video mode: VGA@100
[2025-04-30 11:15:11 UTC][ZED][INFO] [Init]  Serial Number: S/N 35159485
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolo11l_half.engine for TensorRT inference...
[04/30/2025-12:15:12] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[04/30/2025

enter id to track (or leave blank to skip): 
enter id to track (or leave blank to skip): 
enter id to track (or leave blank to skip): 
enter id to track (or leave blank to skip): 
enter id to track (or leave blank to skip): 1


tracking object id 1
horizontal centre: 357
horizontal centre: 357
horizontal centre: 356
horizontal centre: 359
horizontal centre: 359
horizontal centre: 353
horizontal centre: 350
horizontal centre: 349
horizontal centre: 347
horizontal centre: 347
horizontal centre: 351
horizontal centre: 368
horizontal centre: 401
horizontal centre: 432
horizontal centre: 467
horizontal centre: 452
horizontal centre: 368
horizontal centre: 295
horizontal centre: 296
horizontal centre: 270
horizontal centre: 210
horizontal centre: 179
horizontal centre: 218
horizontal centre: 313
horizontal centre: 420
horizontal centre: 423
horizontal centre: 411
horizontal centre: 397
horizontal centre: 362
horizontal centre: 337
horizontal centre: 321
horizontal centre: 282
horizontal centre: 260
horizontal centre: 255
horizontal centre: 244
horizontal centre: 243
horizontal centre: 319
horizontal centre: 382
horizontal centre: 380
horizontal centre: 391
horizontal centre: 393
horizontal centre: 394
horizontal ce

TypeError: 'bool' object is not subscriptable